In [11]:
import numpy as np
import pandas as pd

from energy_consumption.help_functions.get_energy_data import get_data

from energy_consumption.models.QR_mappings_interactions import get_QR_mappings_interactions
from energy_consumption.models.lasso_regression import get_lasso_forecasts
from energy_consumption.models.seasonal_QR_hh import get_seasonal_QR_hourly_holidays

In [3]:
energydata = get_data(set_wed=True, wednesday_morning=True)

100%|██████████| 102/102 [01:07<00:00,  1.51it/s]


In [40]:
# make sure to import energydata before 
model_km = {
    "name": "km",
    "function": get_QR_mappings_interactions,
    "horizon_integration": [True, True, True,False, False, False], 
    "weights": [1/2, 1/2, 1, 0, 0, 0]
}

model_lasso = {
    "name": "lasso",
    "function": get_lasso_forecasts,
    "horizon_integration": [True, True, False, False, False, False],
    "weights": [1/2, 1/2, 0, 0, 0, 0]
}

model_disaggregated = {
    "name": "disagg",
    "function": get_seasonal_QR_hourly_holidays,
    "horizon_integration": [False, False, False, True, True, True],
    "weights": [0, 0, 0, 1, 1, 1]
}

models = [model_km, model_lasso, model_disaggregated]

In [47]:
def get_combined_models(models, energydata=pd.DataFrame, wednesday_morning=False): 
    """
    list of models, each model 
    contains dataframe of forecasting results, 
    list of boolean values of each horizon 
    weights, if boolean value is true 
    """

    if energydata.empty:
        energydata=get_data(wednesday_morning=wednesday_morning)

    # calculate forecasts
    forecasts = {}
    for model in models:
        name = str('forecast_model_'+ model['name'])
        forecasts[name] = model["function"](energydata)

    # create combined data frame
    combined_model = list(forecasts.values())[0].copy()
    combined_model.loc[:, 'q0.025':'q0.975'] = 0
    
    # for each model and horizon: add weighted forecasts, if horizon_integration = True 
    for model in models: 
        name = str('forecast_model_' + model['name'])
        for h in range(6):
            if model['horizon_integration'][h] == True: 
                to_add = model['weights'][h] *np.array(
                    forecasts[name].iloc[h].loc['q0.025':'q0.975'])
                combined_model.loc[combined_model.index[h],
                                   'q0.025':'q0.975'] += to_add
                

    return(combined_model)

In [48]:
combined_model = get_combined_models(models,energydata,wednesday_morning=True)
combined_model

,date_time,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-24 12:00:00,2023-11-29,energy,36 hour,45.126149,54.639109,61.861997,66.144286,73.565996
1,2023-11-24 16:00:00,2023-11-29,energy,40 hour,41.606799,51.919562,58.462558,63.232546,71.523425
2,2023-11-24 20:00:00,2023-11-29,energy,44 hour,38.525501,50.064751,57.688750,60.315999,63.821999
3,2023-11-25 12:00:00,2023-11-29,energy,60 hour,51.465251,55.633250,66.432514,68.429500,71.939999
4,2023-11-25 16:00:00,2023-11-29,energy,64 hour,49.166001,53.268500,62.466750,64.728750,68.112249
5,2023-11-25 20:00:00,2023-11-29,energy,68 hour,50.303001,53.020750,58.618500,61.431750,64.174999
